In [1]:
# Uncomment and run this cell if you're on Colab or Kaggle
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'notebooks'...
remote: Enumerating objects: 515, done.
remote: Counting objects: 100% (515/515), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 515 (delta 245), reused 479 (delta 231), pack-reused 0
Receiving objects: 100% (515/515), 29.39 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (245/245), done.
/kaggle/working/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [2]:
#hide
from utils import *
setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Go to Settings > Accelerator and select GPU.
Using transformers v4.11.3
Using datasets v1.16.1


# Transformer Anatomy

Encoder-decoder architecture of the transformer, with the encoder shown in the upper half of the figure and the decoder in the lower half.

<img alt="transformer-encoder-decoder" caption="Encoder-decoder architecture of the transformer, with the encoder shown in the upper half of the figure and the decoder in the lower half" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_transformer-encoder-decoder.png" id="transformer-encoder-decoder"/>

## The Encoder

The transformer's encoder consists of many encoder layers stacked next to each other. As illustrated in the figure below, each encoder layer receives a sequence of enbeddings and feeds them through the following sublayers:
* A multi-head self-attention layer
* A fully connected feed-forward layer that is applied to each input embedding

> The main role of the encoder stack is to "update" the input embeddings to produce representations that encode some contextual information in the sequence.

<img alt="encoder-zoom" caption="Zooming into the encoder layer" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_encoder-zoom.png" id="encoder-zoom"/>

### Self-Attention

* The main idea behind self-attention is that instead of using a fixed embedding for each token, we can use the whole sequence to compute weighted average of each embedding. In other words, given a sequence of embeddings, self-attention produces a sequence of new embeddings where each new embedding is a linear combination of all original embedding vectors.
* The following diagram shows how self-attention updates raw token embeddings (upper) into contextualized embeddings (lower) to create contextualized representations that incorporate information from the whole sequence.

<img alt="Contextualized embeddings" caption="Diagram showing how self-attention updates raw token embeddings (upper) into contextualized embeddings (lower) to create representations that incorporate information from the whole sequence" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_contextualized-embedding.png" id="contextualized-embeddings"/>

Let's now take a look at how we can calculate the attention weights.

### Scaled dot-product attention

* There are several ways to implement a self-attention layer, but the most common one is scaled dot-product attiontion. It includes four main steps:
1. Project each token embedding into three vectors: key, query, value
2. Compute attention scores
3. Compute attention weights by applying softmax
4. For a specific token, multiply attention weight by value vector to get the updated form of token embedding.

* The following diagram represents the operations to compute scaled dot-product attention. Let's implement it:

<img alt="Operations in scaled dot-product attention" height="125" caption="Operations in scaled dot-product attention" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_attention-ops.png" id="attention-ops"/>

#### 1. Tokensize the text and get input IDs:

In [3]:
# import the tokenizer
from transformers import AutoTokenizer
model_ckpt = "bert-base-uncased"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
# use our tokenizer to get the input IDs
# for simplicity, exclued [CLS] and [SEP] tokens
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

#### 2. Generate Raw Embeddings (non-contextual embedding):
* First, let's instantiate an Embedding Layer. We can do this by using `torch.nn.Embedding` layer that acts as a lookup table for each input ID:

In [5]:
from torch import nn
from transformers import AutoConfig
# load config.json associated with bert-base-uncased checkpoint
config = AutoConfig.from_pretrained(model_ckpt)

# instantiate the embedding layer
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

* Now, we can generate our raw embedding vectors:
> NOTE: These vectors are raw, i.e., non-contextual and the position is NOT considered (no positional encoding)

````python
text = "time flies like an arrow"
````

In [6]:
# feed input IDs into the embedding layer to generate non-contextual embedding
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()
# [batch_size, seq_len, hidden_dim]

torch.Size([1, 5, 768])

In [7]:
inputs_embeds

tensor([[[-0.1104,  2.4377,  0.7857,  ..., -0.6889, -1.6863, -1.3295],
         [-1.0812, -2.0947, -0.3449,  ..., -0.8268,  0.7888,  1.8255],
         [-0.3827,  0.4757, -1.0224,  ...,  1.3848,  1.0920, -0.0045],
         [-1.3329,  1.5028,  0.9517,  ...,  0.2071, -1.2100,  1.7652],
         [ 0.0721, -0.0388, -0.4580,  ...,  2.0672, -0.2770,  1.8852]]],
       grad_fn=<EmbeddingBackward0>)

#### 3.Create query, key, and value vectors:
* Now, let's create the query, key, and value vectors and calculate the attention scores using dot product as the similarity function:
> **NOTE**: For simplicity, projection step for **K**,**Q**, and **V** is negelected for now. That is, **K**, **Q**, and **V** are the same as the **raw embedding vector**. Later we'll generate three variants of this **raw vector** by projecting it using different matrices (The weights of those matrices will changed during training)

> **NOTE 2**: `tensor.size()` returns a tuple-like object of dimensions. `tensor.shape` is an attribute of the tensor whereas `size()` is a function. They both return the same value. `.shape` is an alias for `.size()`, and was added to more closely match numpy.

In [25]:
import torch
from math import sqrt 

# K, Q, V are the same
query = key = value = inputs_embeds # size (1, 5, 768)
query.shape, query.size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

In [26]:
query.size(-1)

768

#### 4.Calculate attention scores:

In [9]:
# calculate attention scores 
dim_k = key.size(-1) # 768
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k) ## takes two batches of matrices and multiplies 
                                                            # each matrix from the first batch with the corresponding matrix in the second batch
scores.size()

torch.Size([1, 5, 5])

In [27]:
key.transpose(1,2).size()

torch.Size([1, 768, 5])

In [28]:
query.size(), key.transpose(1,2).size()

(torch.Size([1, 5, 768]), torch.Size([1, 768, 5]))

In [35]:
scores.size()

torch.Size([1, 5, 5])

In [33]:
 scores.dim()

3

In [36]:
scores

tensor([[[ 2.8306e+01, -1.8534e+00,  2.0952e-01, -1.3421e-01, -1.4915e+00],
         [-1.8534e+00,  2.8446e+01, -1.2458e+00, -2.6085e+00, -1.3103e+00],
         [ 2.0952e-01, -1.2458e+00,  2.9549e+01, -1.1375e-02,  3.3305e-01],
         [-1.3421e-01, -2.6085e+00, -1.1375e-02,  2.6956e+01,  1.3034e-01],
         [-1.4915e+00, -1.3103e+00,  3.3305e-01,  1.3034e-01,  2.8746e+01]]],
       grad_fn=<DivBackward0>)

#### 5. Calculate attention weights by applying the softmax function on the attention scores:

In [37]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.size()#.sum(dim=-1)

torch.Size([1, 5, 5])

#### 6. Update the raw token embedding: mulitply attention weights by the values:

In [13]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

#### Wrap the previous steps in a function:

In [14]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

#### Multi-headed attention

* We just used **embedding** "as is" (in form of key, query, value) to compute the attention scores and weights. In practice, the self-attention layer applies three independent linear transformations to each embedding to generate different variation of the **embedding** in form of *key, query, and value*. These transformation project the **embeddings** and each projection carries its own set of learnable parameters.
* It also beneficial to have multiple sets of linear projections, each one represention a so-called ***attention head***. The resulting **multi-head attention layer** is illustrated below.
> Why we need more than attention head? Each head focuses on one aspect of similarity. For instance, one head can focus on subject-verb interaction, whereas another finds nearby adjectives.

<img alt="Multi-head attention" height="125" caption="Multi-head attention" src="https://raw.githubusercontent.com/nlp-with-transformers/notebooks/48e4a5e5c44b86e1593c0945a49af9675cfd7158//images/chapter03_multihead-attention.png" id="multihead-attention"/>

* In the given Python code, `self.q(hidden_state)` is calling the linear transformation layer `q` with input `hidden_state`. The `q` layer is an instance of `nn.Linear` and is defined in the `__init__` method of the `AttentionHead` class as follows:
````python
self.q = nn.Linear(embed_dim, head_dim)
````
* This layer takes an input tensor of shape `(batch_size, sequence_length, embed_dim)` and applies a linear transformation to it to produce an output tensor of shape `(batch_size, sequence_length, head_dim)`. 
* So when `self.q(hidden_state)` is called in the forward method, it applies the linear transformation to the input `hidden_state` and produces an output tensor of shape `(batch_size, sequence_length, head_dim)` which is used in further calculations to compute the attention scores.

* In the forward pass of the `AttentionHead` class, the input `hidden_state` is used to calculate the scaled dot-product attention for one attention head. * The attention head projects the input `hidden_state` from the original `embed_dim` to a lower `head_dim` using three linear transformation layers: `q`, `k`, and `v`. 
* The output of the attention head is the weighted sum of the input `hidden_state` based on the attention weights calculated by the head.

In [38]:
# create a class for one attention head
class AttentionHead(nn.Module):
    # initialize the class
    def __init__(self, embed_dim, head_dim): # embed_dim=768, head_dim=64 number of dimensions we want to project the hidden state to
        # call the parent class
        super().__init__()
        # create the linear transformation layers
        self.q = nn.Linear(embed_dim, head_dim) # project (1, 5, 768) -> (1, 5, 64)
        self.k = nn.Linear(embed_dim, head_dim) # project (1, 5, 768) -> (1, 5, 64)
        self.v = nn.Linear(embed_dim, head_dim) # project (1, 5, 768) -> (1, 5, 64)

    # define the forward pass
    def forward(self, hidden_state):                         # hidden_state.size() = inputs_embeds.size() = (1, 5, 768) = (batch_size, seq_len, hidden_size) 
        # calculate the scaled dot product attention for one head 
        dim_k = self.k(hidden_state).size(-1)                # get the dimension of the key   (1, 5, 64) -> 64
        scores = torch.bmm(self.q(hidden_state),             # calculate the attention scores (1, 5, 64) * (1, 64, 5) = (1, 5, 5)
                           self.k(hidden_state).transpose(1,2)) / sqrt(dim_k) 
        attn_weights = F.softmax(scores, dim=-1)             # calculate the attention weights (1, 5, 5)
        return torch.bmm(attn_weights, self.v(hidden_state)) # return the attention outputs (1, 5, 5) * (1, 5, 64) = (1, 5, 64)

            

* No that we have a **single attention head**, we can **concatenate** the outputs of each one to implement the full **multi-head attention** layer:

> Notice that the concatenated output from the attention head is also fed through a final linear layer to produce an output tensor of shape `[batch_size, seq_len, hidden_dim]` that is suitable for the feed-forward network downstream.

In [39]:
# create a class for the multi-head attention
class MultiHeadAttention(nn.Module):
    # initialize the class
    def __init__(self, config):
        # call the parent class
        super().__init__()
        
        # create the attention heads
        embed_dim = config.hidden_size         # 768
        num_heads = config.num_attention_heads # 12
        head_dim = embed_dim // num_heads      # 64

        # create the attention heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)] # create 12 attention heads where each K,V,Q is of size 64
        )
        # create the output linear transformation
        self.output_linear = nn.Linear(embed_dim, embed_dim)
        
    # define the forward pass
    def forward(self, hidden_state):           # hidden_state.size() = inputs_embeds.size() = (1, 5, 768) = (batch_size, seq_len, hidden_size)
        # concatenate the attention outputs of the 12 attention heads
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        # apply the output linear transformation
        x = self.output_linear(x)
        return x

In [40]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)    
attn_output.size() 

torch.Size([1, 5, 768])

In [46]:
inputs_embeds.size()

torch.Size([1, 5, 768])

In [43]:
multihead_attn

MultiHeadAttention(
  (heads): ModuleList(
    (0): AttentionHead(
      (q): Linear(in_features=768, out_features=64, bias=True)
      (k): Linear(in_features=768, out_features=64, bias=True)
      (v): Linear(in_features=768, out_features=64, bias=True)
    )
    (1): AttentionHead(
      (q): Linear(in_features=768, out_features=64, bias=True)
      (k): Linear(in_features=768, out_features=64, bias=True)
      (v): Linear(in_features=768, out_features=64, bias=True)
    )
    (2): AttentionHead(
      (q): Linear(in_features=768, out_features=64, bias=True)
      (k): Linear(in_features=768, out_features=64, bias=True)
      (v): Linear(in_features=768, out_features=64, bias=True)
    )
    (3): AttentionHead(
      (q): Linear(in_features=768, out_features=64, bias=True)
      (k): Linear(in_features=768, out_features=64, bias=True)
      (v): Linear(in_features=768, out_features=64, bias=True)
    )
    (4): AttentionHead(
      (q): Linear(in_features=768, out_features=64, bias=

In [44]:
#hide_output
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

<IPython.core.display.Javascript object>

### The Feed-Forward Layer

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [ ]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

### Adding Layer Normalization

<img alt="Transformer layer normalization" height="500" caption="Different arrangements of layer normalization in a transformer encoder layer" src="images/chapter03_layer-norm.png" id="layer-norm"/>

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Apply layer normalization and then copy input into query, key, value
        hidden_state = self.layer_norm_1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state)
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [ ]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

### Positional Embeddings

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, 
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config) 
                                     for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

### Adding a Classification Head

In [ ]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        
    def forward(self, x):
        x = self.encoder(x)[:, 0, :] # select hidden state of [CLS] token
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [ ]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 3])

## The Decoder

<img alt="Transformer decoder zoom" caption="Zooming into the transformer decoder layer" src="images/chapter03_decoder-zoom.png" id="decoder-zoom"/> 

In [ ]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [ ]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[26.8082,    -inf,    -inf,    -inf,    -inf],
         [-0.6981, 26.9043,    -inf,    -inf,    -inf],
         [-2.3190,  1.2928, 27.8710,    -inf,    -inf],
         [-0.5897,  0.3497, -0.3807, 27.5488,    -inf],
         [ 0.5275,  2.0493, -0.4869,  1.6100, 29.0893]]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

### Sidebar: Demystifying Encoder-Decoder Attention

### End sidebar

## Meet the Transformers

### The Transformer Tree of Life

<img alt="Transformer family tree" caption="An overview of some of the most prominent transformer architectures" src="images/chapter03_transformers-compact.png" id="family-tree"/>

### The Encoder Branch

### The Decoder Branch

### The Encoder-Decoder Branch

## Conclusion